# <span style="color:red">Taxi-v3 With Q-Learning</span>

### The Taxi Problem from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition" by Tom Dietterich

## [Taxi-v3](https://gym.openai.com/envs/Taxi-v3/)

In [52]:
import gym
import random 
from IPython import display
import matplotlib.pyplot as plt
import time
import numpy as np
%matplotlib inline

## Observations: 
    There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger <br>
    (including the case when the passenger is in the taxi), and 4 destination locations. 25*5*4 = 500
    
 ##   Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
    
 ##   Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
        
  ##  Actions:
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east 
    - 3: move west 
    - 4: pickup passenger
    - 5: dropoff passenger
    
##    Rewards: 
    There is a reward of -1 for each action and an additional reward of +20 for delivering the passenger.<br>
     There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    


In [53]:
env = gym.make('Taxi-v3')
print("Number of observation space: ", env.observation_space.n)
print("Number of action space: ", env.action_space.n)
env.render()

Number of observation space:  500
Number of action space:  6
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Sample rendering of Taxi-v3 witout training

In [54]:
env.reset()
total_reward = 0
for i in range(50):
    action = env.action_space.sample()
    new_state, reward, done, info = env.step(action);
    total_reward+=reward
    display.clear_output(wait=True)
    time.sleep(.2)
    env.render()

    if done:
        break

print("Total reward without training: ", total_reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Total reward without training:  -149


# Q-learing a Temporal-difference(TD) algorithm

## Step of Q-learing

- 1: First, we initialize the Q function to some arbitrary values.
- 2: We take an action from a state using epsilon-greedy policy ( ) and move it. to the new state 
- 3: We update the Q value of a previous state by following the update rule.
- 4: We repeat the steps 2 and 3 till we reach the terminal state

## Q-Table euqtion:

 $Q(s,a)  = Q(s,a)+ \alpha(r + \gamma  max(Q( s{}', a ))  - Q(s,a))$


### Here
- a = action
- s = observation space
- r = reward
- Q - qtable



In [55]:
'''
Initializing alpha, gamma and epsilon

Initializing Q-talbe : q = size [observation_space, action_space]

'''
alpha =0.4
gamma = 0.999
epsilon = 0.017

q =np.zeros((env.observation_space.n, env.action_space.n))

In [56]:
'''
First calculated the max( Q(s,a) ) here iterate all action of the observation sapce s.

'''
def update_q_table(prev_state, action, reward, nextstate, alpha, gamma):
    qa = max([q[nextstate,a] for a in range(env.action_space.n)])

    q[prev_state, action] += alpha*(reward+gamma*qa-q[prev_state, action])

In [57]:
'''
We generate some random number in uniform distribution and if the number is less than the epsilon,
we explore a different action in the state,
or else we exploit the action that has a maximum q value
'''

def epsilon_greedy_policy(state, epsilon):
    if (random.uniform(0,1)< epsilon):
        return env.action_space.sample()
    else:
        return np.argmax([q[state,a] for a in range(env.action_space.n)])


## Training our Agent.

In [58]:
'''
First  chose a action by our epsilon_greedy_policy
Then take a setp by this action and calculate next state and reward
After that we update our q-table with next stete and reward
'''

env.reset()
for i in range(20000):
    total_reward =0

    prev_state = env.reset()

    while True:
        action = epsilon_greedy_policy(prev_state, epsilon)
        nextstate, reward, done, _ = env.step(action)
        update_q_table(prev_state, action, reward, nextstate,alpha, gamma)

        prev_state = nextstate

        total_reward +=reward

        if done:
            break
    if i%1000 ==0:
        print("Step : {} || Total Reward: {}".format(i,total_reward))


Step : 0 || Total Reward: -587
Step : 1000 || Total Reward: 10
Step : 2000 || Total Reward: 9
Step : 3000 || Total Reward: 5
Step : 4000 || Total Reward: 7
Step : 5000 || Total Reward: 7
Step : 6000 || Total Reward: 10
Step : 7000 || Total Reward: -2
Step : 8000 || Total Reward: 9
Step : 9000 || Total Reward: 7
Step : 10000 || Total Reward: 7
Step : 11000 || Total Reward: 8
Step : 12000 || Total Reward: 10
Step : 13000 || Total Reward: 8
Step : 14000 || Total Reward: 3
Step : 15000 || Total Reward: 10
Step : 16000 || Total Reward: 3
Step : 17000 || Total Reward: 11
Step : 18000 || Total Reward: 7
Step : 19000 || Total Reward: 5


## Now play Taxi-v3 with our q table

- Here we play several episode. 
- Each episode is individual game.
- First Taxi goes in the station where people are that station
- Then took that people in taxi and drop in destination

## Rendering:
    - blue: passenger
    - magenta: destination
    - yellow: empty taxi
    - green: full taxi
    - other letters (R, G, Y and B): locations for passengers and destinations



In [61]:
env.reset()
max_steps = 100
# lets test for 5 episodes
for episode in range(5):
    state = env.reset();
    step = 0;
    done = False;
    total_reward =0
    
    for step in range(max_steps):
        
        #take the action with maximum expected future reward form the q-table
        action = np.argmax(q[state,:]);
        
        new_state, reward, done, info = env.step(action);
        display.display(env.render())
        time.sleep(.25)
        # display.clear_output(wait=True)
        total_reward+=reward
        
        if done:
            print("Episode: ",episode);
            print("Number of steps taken: ",step);
            print("Reward: ", total_reward)
            #env.render();
            time.sleep(3)
            
            break;
        display.clear_output(wait=True)    
        state = new_state;

#close the connection to the environment
env.close();

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


None

Episode:  4
Number of steps taken:  12
Reward:  8
